In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
from datetime import datetime
#import uuid,json,arcpy

In [1]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

········


In [2]:
find_storymaps = gis.content.search("title:Community Storymap*", item_type="Web Mapping Application", max_items=480, outside_org=False)
#Set up a log file to write errors to
#today = [dt.date.today()]
#log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Update Education"
#log_file = os.path.join(log_path, "Update Education_" + str(today[0]) + str(".txt"))
#out = open(log_file,'w')

In [8]:
for f in find_storymaps:
    print(f.title,f.id)
    if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communit') and 'Community' in str(f.title):
        csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
        sections = csmap.properties["values"]["story"]["sections"]
        panel_edu = sections[5]
        community = f.title.replace(" Community Storymap", "").replace(" (Unincorporated Community)", "").replace("'","''")
        if community.startswith("City & Borough of "):
            community = community.replace("City & Borough of ", "")
        if community.startswith("Municipality of "):
            community = community.replace("Municipality of ","")
        print(community)
        query = "PhysicalCity = '" + str(community.replace("'","''")) + "'"

        try:
        ####EDUCATION PANEL####
            ak_schools = FeatureLayer("https://maps.commerce.alaska.gov/arcgis/rest/services/Education_Related/Alaska_Schools/MapServer/0")
            community_sch_lst = ak_schools.query(where=query, out_fields = "*").df
            ##Check to see if there are schools. If not, post statement to panel that there are no open schools in community. 
            ##otherwise, create a schools table.
            if str(community_sch_lst) == 'Empty DataFrame\nColumns: []\nIndex: []':
                print("There are no schools located in " + str(community) + ", or specific school location information is unavailable for this community's schools.")
                panel_edu["content"] = '<p>\n<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n</style>\n</p>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524785467480" data-storymaps-type="navigate">Back to Table of Contents</a><br>\n<style type="text/css">\n</style>\n</p>\n\n<p>&nbsp;</p>\n\n<p>There are no schools currently open in ' + str(community) + ', or school data is unavailable in this community. Many students across the state attend correspondence schools. For a list of correspondence school in Alaska, <a href="https://education.alaska.gov/alaskan_schools/corres/docs/correspondence-school-directory.docx" target="_blank">click here.</a></p><p>&nbsp;</p>\n\n<p>For more information about Alaska schools and education, click <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=cff461c2397d4c66ae2c28dbfb62319f" target="_blank">here</a>.</p>\n'
                csmap.save()
            else:
                sch_distr = community_sch_lst.SchoolDistrict[0]
                schools = community_sch_lst.EntityName
                school_list = ["<table border = '1'><tr><th width='45%'>School</th><th width='45%'>Address</th><th width='10%'>Zip Code</th></tr><tbody>"]
                for s in schools:
                    school_data = ak_schools.query(where="EntityName = '" + str(s.replace("'","''")) + "'",out_fields='*')
                    address = school_data.df.PhysicalAddress[0]
                    city = school_data.df.PhysicalCity[0]
                    zipcode = school_data.df.PhysicalZip[0]
                    school_list.append("<tr><td>" + str(s) + "</td><td>" + address + "</td><td>" + zipcode + '</td></tr>')
                school_list.append("</tbody></table>")
                embed_schools = ' '.join(school_list)

                #Update panel schools listing
                panel_edu["content"] =  '<p>&nbsp;</p>\n\n<p>\n<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n</style>\n</p>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524785467480" data-storymaps-type="navigate">Back to Table of Contents</a><br>\n<style type="text/css">\n</style>\n</p>\n\n<p>&nbsp;</p>\n\n<p>' + str(community) + " schools are part of " + str(sch_distr) + ". </p>\n\n<p>&nbsp;</p><p><i>Click the school icons directly in the map to see enrollment data from 2011 to present for each school. </i></p><p>\n\n<p>&nbsp;</p><p>" + str(embed_schools) + '</p><p>&nbsp;</p>\n\n Many students across the state attend correspondence schools. For a list of correspondence schools in Alaska, <a href="https://education.alaska.gov/alaskan_schools/corres/docs/correspondence-school-directory.docx" target="_blank">click here.</a></p><p>&nbsp;</p>\n\n<p>For more information about Alaska schools and education, click <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=cff461c2397d4c66ae2c28dbfb62319f" target="_blank">here</a>.</p>\n'
                csmap.save()
                print("School table posted to the education panel for " + str(community) + ".")
    
        except Exception as e:
            print("------ISSUE WITH " + str(f.title) + "------" + str(e))
#        out.write("Education panel for " + str(f.title) +  " not properly updated. See storymap for more info.")
#        out.write('\n\n')
#        print("Community written to error logfile.")
#        continue

South Lakes Community Storymap (Unincorporated Community) 7cee4d422e094ffd8d22b83cddac4042
South Lakes
There are no schools located in South Lakes, or specific school location information is unavailable for this community's schools.
Beaver Community Storymap (Unincorporated Community) 4d61d571ea4c4fd98fb41684e4f4998f
Beaver
School table posted to the education panel for Beaver.
Cordova Community Storymap 61c440f6af6e4f4d86063747cebb7f2f
Cordova
School table posted to the education panel for Cordova.
Port Alexander Community Storymap ee85fd02a20f482aa4158ae622e575dc
Port Alexander
School table posted to the education panel for Port Alexander.
Deltana Community Storymap (Unincorporated Community) dfbd9a1914e346fc96b98c4b591e3f60
Deltana
There are no schools located in Deltana, or specific school location information is unavailable for this community's schools.
Whale Pass Community Storymap (Unincorporated Community) f25f15a0411241e2a77fdd4187014086
Whale Pass
School table posted to the e

Buffalo Soapstone
There are no schools located in Buffalo Soapstone, or specific school location information is unavailable for this community's schools.
Unga Community Storymap (Unincorporated Community) 4feb01890ee746a0961414cc46895b4a
Unga
There are no schools located in Unga, or specific school location information is unavailable for this community's schools.
Egegik Community Storymap 3ac87748cd4c4e77a9f8e40f291382c1
Egegik
School table posted to the education panel for Egegik.
Atqasuk Community Storymap b42d7c31c0e642a89577e88ab7bf36ab
Atqasuk
School table posted to the education panel for Atqasuk.
Ouzinkie Community Storymap f2707ae804a84e58b84d991a35c5e544
Ouzinkie
School table posted to the education panel for Ouzinkie.
Aniak Community Storymap 4722232513444b42ae1122f7a1b225cf
Aniak
------ISSUE WITH Aniak Community Storymap------must be str, not NoneType
Alakanuk Community Storymap 56bca62b789a4d908a00803490bf6a96
Alakanuk
School table posted to the education panel for Alakanuk

Port Alsworth Community Storymap (Unincorporated Community) 8be88ef45bec4df190c19113e06dea07
Port Alsworth
School table posted to the education panel for Port Alsworth.
Evansville Community Storymap (Unincorporated Community) 02ab1e339df241bf80476c7de48b3045
Evansville
There are no schools located in Evansville, or specific school location information is unavailable for this community's schools.
Sanak Community Storymap (Unincorporated Community) 673a748c4edb4759993a4ff40424da96
Sanak
There are no schools located in Sanak, or specific school location information is unavailable for this community's schools.
Excursion Inlet Community Storymap (Unincorporated Community) 7d2a5d5f13704c55856cccb4b02cd8dc
Excursion Inlet
There are no schools located in Excursion Inlet, or specific school location information is unavailable for this community's schools.
Dyea Community Storymap (Unincorporated Community) 8404d900ef8e4787b6a3d72a53746e76
Dyea
There are no schools located in Dyea, or specific sc

Iditarod Community Storymap (Unincorporated Community) ec60f3422ddb42f68a1967a03e8df5f2
Iditarod
There are no schools located in Iditarod, or specific school location information is unavailable for this community's schools.
Lowell Point Community Storymap (Unincorporated Community) 2b59848368bd4c7688b923b9dcce4280
Lowell Point
There are no schools located in Lowell Point, or specific school location information is unavailable for this community's schools.
King Cove Community Storymap d0bf043f426f4c2f9b8a05b91873fd08
King Cove
School table posted to the education panel for King Cove.
Telida Community Storymap (Unincorporated Community) 1f2bbad57edd48a89612d43c0b49ab72
Telida
There are no schools located in Telida, or specific school location information is unavailable for this community's schools.
Kodiak Community Storymap 64a5e5a8751e413687ac2e87dde4f887
Kodiak
School table posted to the education panel for Kodiak.
Nunam Iqua Community Storymap ce8dcdc49dea4ee382cf3b3ea58d03d2
Nunam Iq

Wainwright
School table posted to the education panel for Wainwright.
Levelock Community Storymap (Unincorporated Community) 19acdf9a5fa843c6a23eb48d38bb69b4
Levelock
School table posted to the education panel for Levelock.
Primrose Community Storymap (Unincorporated Community) 16195bac8dad4a51bf1ce1b37e1b9eae
Primrose
There are no schools located in Primrose, or specific school location information is unavailable for this community's schools.
Delta Junction Community Storymap b9cae9a278dd47e68a0867ec7be007f8
Delta Junction
School table posted to the education panel for Delta Junction.
Hollis Community Storymap (Unincorporated Community) ce7b21d4eae44352aa15466af42bb56e
Hollis
School table posted to the education panel for Hollis.
Hooper Bay Community Storymap 0b0dbcafc57d4971b4daeb307c6eab15
Hooper Bay
School table posted to the education panel for Hooper Bay.
Gustavus Community Storymap cc996b3d880240b59267d828fbd52900
Gustavus
School table posted to the education panel for Gustavus.

Afognak Community Storymap (Unincorporated Community) 256c027055174f2fbb98b3e5eb71a94a
Afognak
There are no schools located in Afognak, or specific school location information is unavailable for this community's schools.
Thorne Bay Community Storymap b9f6b531813e43f0b1698892ff4e19d0
Thorne Bay
------ISSUE WITH Thorne Bay Community Storymap------must be str, not NoneType
Auke Bay Community Storymap (Unincorporated Community) 9b0df5b6160f4f6293b70d01eb160b21
Auke Bay
There are no schools located in Auke Bay, or specific school location information is unavailable for this community's schools.
Levelock Community Storymap (Unincorporated Community) 19acdf9a5fa843c6a23eb48d38bb69b4
Levelock
School table posted to the education panel for Levelock.
Manokotak Community Storymap a48982af83d44e28b36d2aacaee5cb34
Manokotak
School table posted to the education panel for Manokotak.
Northway Community Storymap (Unincorporated Community) 5295e7ee91c14d50bed2fd20caf55b54
Northway
School table posted to

Metlakatla
School table posted to the education panel for Metlakatla.
Atka Community Storymap ee1d436c6335464782ed357aa326e0e5
Atka
School table posted to the education panel for Atka.
Belkofski Community Storymap (Unincorporated Community) 157addd37f9049bcb0b55e80385e7d6f
Belkofski
There are no schools located in Belkofski, or specific school location information is unavailable for this community's schools.
Shageluk Community Storymap f35b43a8a3cf4798a1ee72d5848f094c
Shageluk
School table posted to the education panel for Shageluk.
Ekuk Community Storymap (Unincorporated Community) 653a0577c827419cb3b6ede0bf878881
Ekuk
There are no schools located in Ekuk, or specific school location information is unavailable for this community's schools.
Nondalton Community Storymap f12270ab470e4fbe9a763921319d9a8b
Nondalton
School table posted to the education panel for Nondalton.
City & Borough of Sitka Community Storymap 2ded44ad6dd4456fbe353f1292e285c2
Sitka
School table posted to the education 

Canyon Village Community Storymap (Unincorporated Community) 54a6b6ac96644a3b9867db21a5afc409
Canyon Village
There are no schools located in Canyon Village, or specific school location information is unavailable for this community's schools.
Chitina Community Storymap (Unincorporated Community) 4f84ec55ecf04d45800a60ca922d3f5f
Chitina
There are no schools located in Chitina, or specific school location information is unavailable for this community's schools.
Tyonek Community Storymap (Unincorporated Community) e1bca51bf19f442c8848c022ed2acbbf
Tyonek
School table posted to the education panel for Tyonek.
Nelson Lagoon Community Storymap (Unincorporated Community) 4ca2a5db94e345aa9c6333731f5ba0e1
Nelson Lagoon
School table posted to the education panel for Nelson Lagoon.
Saint Lawrence Island Community Storymap (Unincorporated Community) d81b6e456ce84f72897f63435a03bf46
Saint Lawrence Island
There are no schools located in Saint Lawrence Island, or specific school location information is

Seldovia
School table posted to the education panel for Seldovia.
Uyak Community Storymap (Unincorporated Community) 1c12c6c857564e6982f40a34c0302728
Uyak
There are no schools located in Uyak, or specific school location information is unavailable for this community's schools.
Tetlin Community Storymap (Unincorporated Community) 2aa129b5298e47ab909cb0d1c0a5ab41
Tetlin
School table posted to the education panel for Tetlin.
Ruby Community Storymap f0a9a0b5e6aa44a18d9dec735de4626a
Ruby
School table posted to the education panel for Ruby.
Meadow Lakes Community Storymap (Unincorporated Community) 339baf014c734cdfb3dbc5eac7cce895
Meadow Lakes
There are no schools located in Meadow Lakes, or specific school location information is unavailable for this community's schools.
Koyuk Community Storymap d29a7e9fd7b54f1f97d71364e26a1324
Koyuk
School table posted to the education panel for Koyuk.
Tok Community Storymap (Unincorporated Community) 2d60770ccd45411186bf78d22f06d867
Tok
School table poste

Cohoe
There are no schools located in Cohoe, or specific school location information is unavailable for this community's schools.
Tanana Community Storymap 70f74d3f3d5a417992597f9867eb1a1d
Tanana
School table posted to the education panel for Tanana.
Edna Bay Community Storymap 4af894fd91be48179c4cf40068d4c65e
Edna Bay
School table posted to the education panel for Edna Bay.
Willow Creek Community Storymap (Unincorporated Community) 25377afbeb854210a170757f0fae94b4
Willow Creek
There are no schools located in Willow Creek, or specific school location information is unavailable for this community's schools.
Deering Community Storymap 52e92b70be5b4d82a061a4a1231d6d98
Deering
School table posted to the education panel for Deering.
Iliamna Community Storymap (Unincorporated Community) df0d661d7c68469b91dcd69c13feb13b
Iliamna
School table posted to the education panel for Iliamna.
Farmers Loop Community Storymap (Unincorporated Community) b8cbdc69afdd4e69b036bf2bf6ea2ad5
Farmers Loop
There 

In [ ]:
### Petersburg
### Haines
### Anchorage
### Wasilla